In [19]:
from PIL import Image
import pytesseract

# Set the path to the Tesseract executable (if necessary)
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"   # Update this path if needed

# Path to your Sindhi image
image_path = r'C:\Users\ASUS\Desktop\line-finetuning\Zeer-Zabar.png'  # Replace with the path to your image
img = Image.open(image_path)

# Perform OCR using Sindhi language data (snd)
text_sindhi = pytesseract.image_to_string(img, lang='ambile_33.720_285_400')  # Using Sindhi trained data
print("Extracted text (Sindhi):")
print(text_sindhi)


Extracted text (Sindhi):
ييت . ۔
سانوڻ جي مَُنڌ گذري چَڪِي آهي,
هاڻي بَڊو شرو ع ٿي ويو آهي. سنڌ
۾ مِينهن گهٽجي ويو هي. هن
سانوتِيءَ ۾ سٺو مِينهن پيو ۽ ۽ درياھ
پر گهڻو چاڙھ آهي.



In [ ]:

# below  mmodel is trained on wordnet_corpus data 

# # # ambile_36.991_229_300  low iteration             (best model) file size = 11571 kb   BCER eval=28.157, BWER eval=58.133



# # ambile_8.175_2763_7900  highest iteration


In [ ]:
# below model is trained on line data 
# ambile-model_31.133_100_100 low iteration 
# ambile-model_8.886_3874_4400
 

Calculating the WER  and CER 


In [2]:
#!/usr/bin/env python3
import os, re, subprocess

# === CONFIG ===
MODEL_NAME  = "ambile-model"
PROJECT_DIR = r"C:\Users\ASUS\Desktop\line-finetuning\tesstrain"
GIT_BASH    = r"C:\Program Files\Git\bin\bash.exe"
ITERATIONS  = 1  # you can bump this if needed

def to_bash_path(p: str) -> str:
    """Convert C:\\foo\\bar -> /c/foo/bar for Git Bash."""
    p = p.strip()
    if re.match(r"^[A-Za-z]:\\", p):
        drive = p[0].lower()
        rest  = p[2:].replace("\\", "/")
        return f"/{drive}{rest}"
    return p.replace("\\", "/")

def run_bash(cmd: str, cwd_win: str = None):
    """Run a single command string in Git Bash (-lc ...)."""
    if cwd_win:
        cwd_bash = to_bash_path(cwd_win)
        cmd = f'cd "{cwd_bash}" && {cmd}'
    print(f"\n> RUN (bash): {cmd}\n")
    p = subprocess.Popen(
        [GIT_BASH, "-lc", cmd],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        encoding="utf-8",
        errors="replace"
    )
    lines = []
    for line in p.stdout:
        print(line, end="")
        lines.append(line)
    p.wait()
    print(f"Process finished with return code {p.returncode}")
    return p.returncode, "".join(lines)

def extract_command(output: str):
    """Return a follow-up command if the Makefile prints one."""
    m = re.search(r"^EVAL_CMD:\s*(.+)$", output, flags=re.M)
    if m:
        return m.group(1).strip()
    time_lines = re.findall(r"^time\s+.+$", output, flags=re.M)
    if time_lines:
        return time_lines[-1].strip()
    py_lines = re.findall(r"^(?:[A-Z_]+=.*\s+)*python\s+.+$", output, flags=re.M)
    if py_lines:
        return py_lines[-1].strip()
    return None

def main():
    # Quick visibility: show where we are and that the GT dir exists (from bash’s POV)
    gt_dir = os.path.join(PROJECT_DIR, f"data\\{MODEL_NAME}-ground-truth")
    bash_gt_dir = to_bash_path(gt_dir)
    _ = run_bash(f'pwd; echo "GT dir (bash view): {bash_gt_dir}"; ls -ld "{bash_gt_dir}" || true', cwd_win=PROJECT_DIR)

    for i in range(1, ITERATIONS + 1):
        print(f"\n================ ITERATION {i}/{ITERATIONS} ================\n")
        # IMPORTANT: run make *inside Git Bash* and force bash for recipe shell.
        make_cmd = f'make -e SHELL=/usr/bin/bash evaluation MODEL_NAME={MODEL_NAME}'
        rc, out = run_bash(make_cmd, cwd_win=PROJECT_DIR)

        if rc != 0:
            print("make failed; not attempting a follow-up command this iteration.")
            continue

        follow = extract_command(out)
        if not follow:
            print("No runnable follow-up command detected in make output.")
            continue

        _ = run_bash(follow, cwd_win=PROJECT_DIR)

    print("\nAll iterations complete.")

if __name__ == "__main__":
    main()



> RUN (bash): cd "/c/Users/ASUS/Desktop/line-finetuning/tesstrain" && pwd; echo "GT dir (bash view): /c/Users/ASUS/Desktop/line-finetuning/tesstrain/data/ambile-model-ground-truth"; ls -ld "/c/Users/ASUS/Desktop/line-finetuning/tesstrain/data/ambile-model-ground-truth" || true

/c/Users/ASUS/Desktop/line-finetuning/tesstrain
GT dir (bash view): /c/Users/ASUS/Desktop/line-finetuning/tesstrain/data/ambile-model-ground-truth
drwxr-xr-x 1 ASUS 197121 0 Sep  3 13:25 /c/Users/ASUS/Desktop/line-finetuning/tesstrain/data/ambile-model-ground-truth
Process finished with return code 0

================ ITERATION 1/1 ================


> RUN (bash): cd "/c/Users/ASUS/Desktop/line-finetuning/tesstrain" && make -e SHELL=/usr/bin/bash evaluation MODEL_NAME=ambile-model

You are using make version: 4.4.1
powershell -Command "Measure-Command { \
	lstmeval  \
		--verbosity=0 \
		--model data/ambile-model/tessdata_best/ambile-model_10.302_2589_2800.traineddata \
		--eval_listfile data/ambile-model/list.

In [3]:
pip install jiwer Levenshtein


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 22.1 MB/s  0:00:00

   ---------------------------------------- 0/4 [rapidfuzz]
   ---------------------------------------- 0/4 [rapidfuzz]
   ---------- ----------------------------- 1/4 [click]
   ---------- ----------------------------- 1/4 [click]
   ------------------------------ --------- 3/4 [jiwer]
   ---------------------------------------- 4/4 [jiwer]

Note: you may need to restart the kernel to use updated packages.
